# Algoritmo de Smith Waterman

## Introdução

Este é um algoritmo de alinhamento local que vai produzir diversos alinhamentos locais possiveis para duas sequências biológicas fornecidas.
As etapas principais deste metodo são a criação de uma matrix S, a identificação dos indices dos valores máximos e finalmente a realização do *traceback* apartir desses indexes.

## Funcionamento

Para efeitos de demonstração serão usadas as sequências utilizadas nas aulas de AASB:

- sequência a: PHSWG
- sequência b: HGWAG

Além disso temos que cada ponto da matriz terá como suas coordenadas index[a][b],

Finalmente assumimos que estamos a lidar com uma penalização por espaços linear.

### Formação de matriz S

A Ideia por detrás do calculo de cada ponto da matrix é simples. Para cada ponto é determinado o maior valor entre as seguintes 4 opções:

- Valor imediatamente a cima ( index[a-1][b] ) mais o valor de penalização de um espaço
- Valor da diagonal ( index[a-1][b-1] ) mais o valor da score da substituição dos dois aminoácidos
- Valor da esquerda ( index[a][b-1] ) mais o valor de penalização de um espaço
- 0, este impede a existência de números negativos e indica o reinicio do alinhamento.

Como esquematizado a seguir:

| D | C |
|---|---|
| E | 0 |

Este conjunto de regras acaba por não ser aplicavél nas 3 seguintes situações:

- O valor de index[0][0] que como não tem D,C ou E e como representa o alinhamento de dois espaços, ou seja nada é 0
- A primeira linha onde como não existe diagonal ou cima, e então deve o máximo entre o valor da *esquerda + espaço* e 0, sendo normalmente 0.
- A primeira coluna onde como não existe diagonal ou esquerda, e então deve o máximo entre o valor de *cima + espaço* e 0, sendo normalmente 0.

* a matriz apresenta as coordenadas [a][b] como nº de linha e coluna, não confundir com os valores calculados*

In [9]:
import sequencinator.aln_sw
import sequencinator.aln as aln
import sequencinator.matrix_tools
import pandas as pd

seq_a ="PHSWG"
seq_b ="HGWAG"
espaco = -8

sw_matrix = sequencinator.aln_sw(seq_a=seq_a,seq_b=seq_b,spc_cost=espaco)
pd.DataFrame(sw_matrix)


,0,1,2,3,4,5
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,8,0,0,0,0
3,0,0,8,0,1,0
4,0,0,0,19,11,3
5,0,0,6,11,19,17


### Determinação dos máximos e os seus indices

Inicialmente deve ser determinado o valor máximo da matriz, que no exemplo anterior é 19

In [10]:
print(sequencinator.matrix_tools.max_matrix(sw_matrix))

19


De seguidade devem ser determinados os indexes da matriz que cujo valor seja igual a este.

***Nota: Na sessão de estudo foi de forma errada afirmada que bastava determinar o ultimo valor máximo***

In [11]:
print(sequencinator.matrix_tools.find_all_max(sw_matrix))

[(4, 3), (5, 4)]


### *Traceback*

Chegou agora o momento de fazer o traceback e que nos vai fornecer os alinhamentos locais possiveis em que o mérito individual é igual ao ao mérito máximo (como determinado no ponto anterior)

A função de traceback irá determinar por ordem de desempate se é possivél chegar ao valor da célula Diagonal, Cima ou Esquerda pelo processo reverso aquele do primeiro ponto. caso o valor da célula seja 0 então irá parar o traceback.

In [12]:
sw_matrix_origin = sequencinator.aln_sw_origin(seq_a=seq_a,seq_b=seq_b,spc_cost=espaco)
pd.DataFrame(sw_matrix_origin)

print("Nesta matriz R indica que o alinhamento foi reinicado ou seja o valor é 0")

,0,1,2,3,4,5
0,R,R,R,R,R,R
1,R,R,R,R,R,R
2,R,D,R,R,R,R
3,R,R,D,R,D,R
4,R,R,R,D,E,E
5,R,R,D,C,D,D


Assim que determinda a origem, e sendo uma operação dinámica, a função deverá determinar a 'origem' das origens até parar.

Esta operação é por natureza iterativa (implementada por compreensão lista por exemplo) para uma lista de todos os indexes determindados no ponto anterior.

In [21]:
alns = sequencinator.aln_sw_traceback(seq_a=seq_a,seq_b=seq_b,spc_cost=espaco)

# fancy print function
for aln in range(len(alns)):
    print("Alinhamento {0}".format(aln+1))
    print(alns[aln][0])
    print(alns[aln][1])

Alinhamento 1
HSW
HGW
Alinhamento 2
HSWG
HGWA
